## PyPDFLoader로 pdf 읽기

### data에 있는 모든 문서 추출하기
- chroma_store가 없을 경우 data 폴더에 있는 pdf 파일들을 읽어서 벡터 DB에 저장 하기 위한 과정

### 청킹 방법 선택하기

In [30]:
# 청킹 방법
splitter_name = "recursive" # recursive, semantic

# 임베딩 모델
MODEL = "text-embedding-3-large" # text-embedding-3-large

# LLM 모델
LLM_MODEL = "gpt-4o-mini" # gpt-4o-mini, openai/gpt-oss-120b

In [31]:
if splitter_name == "recursive" :
    # recursive_character_text_splitter로 구분자 재귀 청킹 기법
    from langchain_text_splitters import RecursiveCharacterTextSplitter

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100
    )
elif splitter_name == "semantic":
    import os
    from langchain_experimental.text_splitter import SemanticChunker
    from langchain_openai.embeddings import OpenAIEmbeddings
    from dotenv import load_dotenv

    load_dotenv()

    # 의미 기반으로 청킹을 하기 위해 OpenAI의 임베딩 모델을 사용
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
    embedding = OpenAIEmbeddings(model=MODEL, api_key=OPENAI_API_KEY)

    text_splitter = SemanticChunker(embedding)

print(text_splitter)

### Debug 함수

In [ ]:
def debug_chunkinfo_aftersplit(all_splits):
    for i, split in enumerate(all_splits):
        print(f"Chunk {i+1}:")
        print(split.page_content)
        print("-" * 40)

### 추출 하기

In [17]:
import os
import glob
from langchain_community.document_loaders import PyPDFLoader

# PDF 파일을 읽어서 텍스트 데이터 추출 및 청킹
def extract_documents_from_pdf(pdf_path):
    # PDF 파일을 읽어서 텍스트 데이터 추출
    loader = PyPDFLoader(pdf_path)
    data_nyc = loader.load()

    # 추출된 텍스트 데이터를 청킹
    splits = text_splitter.split_documents(data_nyc)
    # debug_chunkinfo_aftersplit(splits) 

    # recursive_character_text_splitter의 경우, 청크가 겹치는 부분이 없으면 연결하지 않음
    # 따라서, 청크가 겹치는 부분이 없을 때는 직접 연결하여 오버랩을 만듦
    # 만약 청크가 겹치는 부분이 있다면, 그 부분은 자동으로 연결됨
    # 여기서는 청크가 겹치지 않는 경우에만 오버랩을 추가함

    # 만약 첫 번째 청크의 끝과 두 번째 청크의 시작이 겹치지 않는다면,
    if splitter_name == "recursive" and (splits[0].page_content[-100:] == splits[1].page_content[:100]):
        print(splits[0].page_content)
        print("----")
        print(splits[1].page_content)
        for i in range(len(splits) - 1):
            splits[i].page_content += "\n" + splits[i + 1].page_content[:50]

    return splits


current_dir = os.getcwd() # 현재 폴더 경로
folder_path = os.path.join(current_dir, "data") # data 폴더 경로 설정
pdf_files = glob.glob(os.path.join(folder_path, "*.pdf")) # PDF 파일 목록 가져오기

all_splits = []
print(f"현재 청킹 방법 : {splitter_name}")
print(f"현재 모델 : {MODEL}")
print(f"PDF 파일 개수: {len(pdf_files)}")
for pdf_file in pdf_files:
    pdf_path = pdf_file
    # temp_docs, merged_path = extract_documents_from_pdf(pdf_file)
    all_splits.extend(extract_documents_from_pdf(pdf_file))

print(f"전체 청크 개수: {len(all_splits)}")

현재 청킹 방법 : semantic
현재 모델 : text-embedding-3-large
PDF 파일 개수: 3
전체 청크 개수: 118


## 텍스트를 벡터로 변환하기
- 임베딩 모델 설정하기

In [32]:
from langchain_openai import OpenAIEmbeddings

from dotenv import load_dotenv

import os
load_dotenv()

if splitter_name != "semantic":
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

    # text-embedding-3-small이 비용 대비 성능이 매우 뛰어나지만, 한글 문서에서는 text-embedding-3-large가 더 나은 성능을 보여 large로 선택.
    MODEL = "text-embedding-3-large"
    embedding = OpenAIEmbeddings(model=MODEL, api_key=OPENAI_API_KEY)

# 예시
# v = embedding.embed_query("뉴욕의 온실가스 저감정책은 뭐야?")
# print(v)
# print(len(v))

## 크로마 DB 생성하고 데이터 불러오기


- 최초에 chrom_store을 만들 때 data에 있는 pdf를 사용해서 생성할 때.

In [33]:
from langchain_chroma import Chroma
import tiktoken
import os
import shutil
import time

persist_directory = './chroma_store' + '_' + splitter_name + '_' + MODEL.replace('.', '_')

encoding = tiktoken.encoding_for_model(MODEL)

TOKEN_LIMIT_PER_BATCH = 40000  # 적절한 토큰 제한

def batch_save(vectorstore, splits):
    current_batch = []
    current_tokens = 0

    for doc in splits: 
        tokens = len(encoding.encode(doc.page_content))
        
        if current_tokens + tokens > TOKEN_LIMIT_PER_BATCH:
            try:
                vectorstore.add_documents(current_batch)
                print(f"✅ {len(current_batch)}개 문서 배치 저장 완료")
            except Exception as e:
                print(f"❌ 배치 저장 중 오류 발생: {e}")
            current_batch = [doc]
            current_tokens = tokens
        else:
            current_batch.append(doc)
            current_tokens += tokens

    # 마지막 배치 처리
    if current_batch:
        vectorstore.add_documents(current_batch)
        print(f"✅ 마지막 배치 {len(current_batch)}개 문서 저장 완료")
    
    return vectorstore


if not os.path.exists(persist_directory):
    print(f"Creating new Chroma store at {persist_directory}...")    
    vectorstore = Chroma(
        embedding_function=embedding,
        persist_directory=persist_directory
    )
    vectorstore = batch_save(vectorstore, all_splits)
else :
    # 이미 존재하는 Chroma store를 불러오기
    print("Loading existing Chroma store...")
    vectorstore = Chroma(
        embedding_function=embedding,
        persist_directory=persist_directory
    )
    # 만약 새로 추가할 PDF 파일이 있다면 데이터 추출 및 저장후, extra_data 폴더에서 data 폴더로 이동
    src_folder = 'extra_data'
    dst_folder = 'data'

    current_dir = os.getcwd() # 현재 폴더 경로
    folder_path = os.path.join(current_dir, src_folder) # extra_data 폴더 경로 설정
    pdf_files = glob.glob(os.path.join(folder_path, "*.pdf")) # PDF 파일 목록 가져오기

    extra_splits = []
    if not pdf_files:
        print("추가할 PDF 파일이 없습니다.")
    else:
        print(f"추가할 PDF 파일 개수: {len(pdf_files)}")
        for pdf_file in pdf_files:
            print(f"Processing {pdf_file}...")
            extra_splits.extend(extract_documents_from_pdf(pdf_file))
            time.sleep(1)

        vectorstore = batch_save(vectorstore, extra_splits)

        # extra_data 폴더의 모든 파일 중 .pdf만 이동
        for filename in os.listdir(src_folder):
            if filename.lower().endswith('.pdf'):
                src_path = os.path.join(src_folder, filename)
                dst_path = os.path.join(dst_folder, filename)
                if os.path.isfile(src_path):
                    shutil.move(src_path, dst_path)

Loading existing Chroma store...
추가할 PDF 파일이 없습니다.


## LLM 설정 후 질문 및 답변

In [23]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import ChatOpenAI
from openai import OpenAI

if LLM_MODEL == "gpt-4o-mini":
    chat = ChatOpenAI(model=LLM_MODEL)
# elif LLM_MODEL == "gpt-oss-120b":
#     HF_API_KEY = os.getenv("HF_API_KEY")
#     chat = ChatOpenAI(
#         base_url="https://router.huggingface.co/v1",
#         api_key=HF_API_KEY,
#         model = LLM_MODEL
#     )

question_answering_promt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Please write your answer in a markdown table format with the main points. Be sure to include all your source and page numbers like (3 ~ 10) in your answer. If you have over one source, you should include all of them. Answer in Korean. \n#Example Format: \n(brief summary of the answer) \n (table) \n  (detailed answer to the question) \n**출처** \n- (file source) (page source and page number) (Please write the quoted text within 20 characters and follow it with ... )\n #Context: {context}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_promt)

- meta data에 있는 정보는 llm이 알 수 없으므로 출처 filename을 포함시켜서 새로운 Document 타입을 생성

In [24]:
import os
from langchain.schema import Document

# Document type인 docs가 context로 넘겨졌을 때 llm은 content에 있는 정보 기반으로 답을 한다.
# 따라서 meta data에 있는 정보는 llm이 알 수 없으므로 출처 filename을 포함시켜서 새로운 Document 타입을 생성한다.
def format_docs_with_source_as_documents(docs):
    new_docs = []
    for d in docs:
        filename = os.path.basename(d.metadata.get("source", ""))
        # 기존 page_content 뒤에 출처 붙이기
        new_content = f"{d.page_content}\n출처: {filename}"

        # 새 리스트 생성 (metadata 유지)
        new_docs.append(
            Document(page_content=new_content, metadata=d.metadata)
        )
    return new_docs



### 질문 입력 및 답변 생성

In [25]:
retriever = vectorstore.as_retriever(k=3)
question = "우리 부부의 연소득은 총 합 6800만원이야. 받을 수 있는 대출은 뭐가 있을까?"
docs = retriever.invoke(question)
# print(type(docs))
# print(docs)
formatted_context = format_docs_with_source_as_documents(docs)

# for d in formatted_context:
#     # print(d.metadata)
#     print(d)
#     print("-" * 40)
# for d in docs:
#     print(d.metadata)
#     print(d.page_content)
#     print("-" * 40)


- openai/gpt-oss-120b 모델 사용

In [ ]:

from openai import OpenAI
from langchain_openai import ChatOpenAI

load_dotenv()
HF_API_KEY = os.getenv("HF_API_KEY")
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=HF_API_KEY
)
# print(formatted_context)
summary_completion = client.chat.completions.create(
    model="openai/gpt-oss-120b",
    messages=[{
        "role": "user", 
        "content": f"You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. you should user performed_context. If the 'perforemd_context' is null, you just say 'it is empyt'. Please keep the answer under 100 characters. Please write your answer in a markdown table format with the main points. Be sure to include all your source and page numbers like (3 ~ 10) in your answer. If you have over one source, you should include all of them. Answer in Korean. \n#Example Format: \n(brief summary of the answer) \n (table) \n  (detailed answer to the question) \n**출처** \n- (file source) (page source and page number) (Please write the quoted text within 20 characters and follow it with ... )\n #Context: {formatted_context}. The question is : 우리 부부의 연소득은 총 합 6800만원이야. 받을 수 있는 대출은 뭐가 있을까?"
        }],
)
print("질문:", question)
print("답변:")
print("-" * 40)
print(summary_completion.choices[0].message.content)

print("\n청킹 방법", splitter_name)
print("임베딩 모델", MODEL)
print("LLM 모델", "openai/gpt-oss-120b")


질문: 우리 부부의 연소득은 총 합 6800만원이야. 받을 수 있는 대출은 뭐가 있을까?
답변:
----------------------------------------
**요약**  
연소득 6,800만원(5% 차감 후 6,460만원)은 첫‑주택구입자라면 `소득추정 70 백만원` 한도 이내이므로 디딤돌대출 신청 가능. 담보주택당 대출한도 2 억원(첫‑주택 2.4 억원) 내에서 10‑30 년 만기, 거치 1 년(또는 비거치) 조건으로 선택할 수 있다. 다만, 소득 60 백만원 초과(첫‑주택 70 백만원 초과) 경우는 취급 불가이니, 첫‑주택 여부를 확인해야 한다.  

| 구분 | 주요 내용 | 비고 |
|------|-----------|------|
| **소득조건** | 5% 차감 후 추정소득 ≤ 5천만원(일반) / ≤ 70백만원(생애최초 주택구입자) | (page 24) |
| **대출한도** | 담보주택당 2 억원 (생애최초 2.4 억원, 신혼·다자녀·2자녀 3.2 억원) | (page 13) |
| **만기·거치** | 10·15·20·30 년, 거치 1 년 또는 비거치 | (page 13) |
| **상환방식** | 원리금 균등·원금 균등·체증식(40세 미만 고정금리) | (page 13) |
| **유의사항** | 첫‑주택이 아닌 경우 소득 60백만원 초과 시 취급 불가 | (page 24) |

**출처**  
- 디딤돌대출_업무처리기준.pdf (P 24) “5% 차감…70백만원…취급 불가”  
- 디딤돌대출_업무처리기준.pdf (P 13) “담보주택 당 한도…대출만기·거치·상환방식”  

청킹 방법 recursive
임베딩 모델 text-embedding-3-large
LLM 모델 openai/gpt-oss-120b


In [35]:
from langchain.memory import ChatMessageHistory

chat_history = ChatMessageHistory()

chat_history.add_user_message(question)

answer = document_chain.invoke(
    {
        "messages": chat_history.messages,
        "context": formatted_context,
    }
)

chat_history.add_ai_message(answer)
print("질문:", question)
print("답변:")
print("-" * 40)
print(answer)

print("\n청킹 방법", splitter_name)
print("임베딩 모델", MODEL)
print("LLM 모델", LLM_MODEL)

질문: 우리 부부의 연소득은 총 합 6800만원이야. 받을 수 있는 대출은 뭐가 있을까?
답변:
----------------------------------------
대출 가능성을 요약하면 다음과 같습니다. 

| 주요 내용               | 세부 사항                                     |
|----------------------|--------------------------------------------|
| 연소득               | 6800만원                                    |
| 대출한도 — 일반      | 2억원 이내                                   |
| 대출한도 — 생애최초  | 2.4억원                                     |
| 대출한도 — 신혼·다자녀 | 3.2억원                                     |
| 소득 추정 기준       | 5천만원 초과 시 대출 취급 불가             |

부부의 연소득이 총 6800만원인 경우, 대출이 가능한 범위는 다음과 같습니다. 일반적인 대출한도는 주택당 최대 2억원이며, 생애 최초 주택 구입자나 신혼·다자녀 가구의 경우 2.4억원 또는 3.2억원까지 대출이 가능합니다. 하지만 소득이 5천만원을 초과하므로 추가 대출 취급은 불가할 수 있습니다. 

**출처** 
- 디딤돌대출_업무처리기준.pdf (22페이지) "소득추정 금액이 60백만원 초과 시 ..." 

청킹 방법 recursive
임베딩 모델 text-embedding-3-large
LLM 모델 gpt-4o-mini


### context 사용 없이 질문을 받았을 경우

In [44]:

from openai import OpenAI
from langchain_openai import ChatOpenAI

load_dotenv()
HF_API_KEY = os.getenv("HF_API_KEY")
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=HF_API_KEY
)
# print(formatted_context)
summary_completion = client.chat.completions.create(
    model="openai/gpt-oss-120b",
    messages=[{
        "role": "user", 
        "content": f"You are an assistant for question-answering tasks. Please keep the answer under 500 characters. Please write your answer in a markdown table format with the main points.  Answer in Korean. \n#Example Format: \n(brief summary of the answer) \n (table) \n  (detailed answer to the question) \n**출처** \n- (file source) (page source and page number and correct full url) (Please write the quoted text within 20 characters and follow it with ... )\n The question is : 우리 부부의 연소득은 총 합 6800만원이야. 받을 수 있는 대출은 뭐가 있을까?"
        }],
)
print("질문:", question)
print("답변:")
print("-" * 40)
print(summary_completion.choices[0].message.content)

print("\n청킹 방법", "context 사용 없음")
print("임베딩 모델", "context 사용 없음")
print("LLM 모델", "openai/gpt-oss-120b")

질문: 우리 부부의 연소득은 총 합 6800만원이야. 받을 수 있는 대출은 뭐가 있을까?
답변:
----------------------------------------
**핵심 요약**  
연소득 6,800만원이면 주택·전세·신용·보증대출 등 4가지 유형을 주로 고려할 수 있습니다.  

| 대출 유형 | 한도(예시) | 금리(연) | 주요 조건 |
|---|---|---|---|
| 주택담보대출 | 연소득 × 5~7배 | 3.1%~4.5% | 소득·신용·부동산 담보 |
| 전세자금대출 | 연소득 × 4배 | 2.8%~4.0% | 전세보증금·소득·신용 |
| 정책보증대출(주택도시보증) | 연소득 × 6배 | 2.5%~3.5% | 보증기관·소득·신용 |
| 신용대출(소액) | 연소득 × 2배 | 4.0%~7.0% | 신용점수·소득 |

**세부 안내**  
- **주택담보**는 가장 높은 한도와 낮은 금리를 제공하지만 부동산 소유가 전제됩니다.  
- **전세자금 대출**은 전세보증금을 담보로 하며, 초기 자금이 부족할 때 활용합니다.  
- **보증대출**은 주택도시보증공사(HUG) 등이 보증해 주어 신용보강이 되며, 소득·주거 안정성에 따라 한도가 결정됩니다.  
- **신용대출**은 담보가 없고 신용점수에 따라 금리가 다소 높으니, 급한 자금이 필요할 때 보조적으로 이용합니다.  

**출처**  
- 금융감독원, “가계대출 한도·금리 안내”, 2023년, https://www.fss.or.kr/... (인용: “연소득 × …”)  
- 주택도시보증공사, “보증대출 상품개요”, 2024, https://www.hug.or.kr/... (인용: “보증·연소득 × 6…”)  
- 한국은행, “주택·전세자금 대출 현황”, 2023, https://www.bok.or.kr/... (인용: “전세자금 연소득 × 4…”)  

청킹 방법 context 사용 없음
임베딩 모델 context 사용 없음
LLM 모델 openai/gpt-oss-120b


## 질의 확장 구현
- 챗본으로 만들 경우 첫번째 질문을 이어 받아서 두번째 질문도 답변을 해야하니 질의 확장에 대한 기능이다. (필수 x)

In [37]:
from langchain_core.output_parsers import StrOutputParser

query_for_other = "3천만원 일때는?"

query_augmentation_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "기존의 대화 내용을 활용하여 사용자가 질문한 의도를 파악해서 한 문장의 명료한 질문으로 변환하라. 대명사나 이, 저, 그와 같은 표현을 명확한 명사로 표현하라. : \n\n{query}",
        ),
    ]
)
query_augmentation_chain = query_augmentation_prompt | chat | StrOutputParser()

augmented_query = query_augmentation_chain.invoke(
    {
        "messages": chat_history.messages,
        "query": query_for_other,
    }
)

print(augmented_query)

부부의 연소득이 3000만원일 때 받을 수 있는 대출은 무엇인가요?


In [40]:
docs_other = retriever.invoke(augmented_query)
# print(type(docs))
# print(docs)
formatted_context_other = format_docs_with_source_as_documents(docs_other)

chat_history.add_user_message(query_for_other)

answer = document_chain.invoke(
    {
        "messages": chat_history.messages,
        "context": formatted_context_other,
    }
)

chat_history.add_ai_message(answer)

print("질문:", query_for_other)
print("답변:")
print("-" * 40)
print(answer)

print("\n청킹 방법", splitter_name)
print("임베딩 모델", MODEL)
print("LLM 모델", LLM_MODEL)

질문: 3천만원 일때는?
답변:
----------------------------------------
연소득이 3000만원일 때의 대출 가능성은 다음과 같이 요약할 수 있습니다.

| 주요 내용               | 세부 사항                                     |
|----------------------|--------------------------------------------|
| 연소득               | 3000만원                                    |
| 대출한도 — 일반      | 2억원 이내                                   |
| 대출한도 — 생애최초  | 2.4억원                                     |
| 대출한도 — 신혼·다자녀 | 3.2억원                                     |
| 소득 추정 기준       | 5천만원 이하 대출 취급 가능                 |

부부의 연소득이 3000만원인 경우, 일반적인 대출한도는 최대 2억원입니다. 또한 생애 최초 주택 구입자나 신혼 및 다자녀 가구의 경우 각각 2.4억원과 3.2억원까지 대출 가능성이 있습니다. 소득이 5천만원 이하이므로 대출 신청이 가능하다는 점 또한 주요합니다.

**출처**
- 디딤돌대출_업무처리기준.pdf (22페이지) "소득추정 금액이 60백만원 이하 ..." 

청킹 방법 recursive
임베딩 모델 text-embedding-3-large
LLM 모델 gpt-4o-mini
